# AI Error Patterns

## Retry with Exponential Backoff

In [ ]:
import asyncio
import logging

logger = logging.getLogger(__name__)

async def retry_with_backoff(
    func,
    max_retries: int = 3,
    base_delay: float = 1.0
):
    '''Retry function with exponential backoff.'''
    for attempt in range(max_retries):
        try:
            return await func()
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            
            delay = base_delay * (2 ** attempt)
            logger.warning(f'Attempt {attempt + 1} failed: {e}. Retrying in {delay}s')
            await asyncio.sleep(delay)

# Test
async def flaky_api_call():
    '''Simulates flaky API.'''
    import random
    if random.random() < 0.7:
        raise Exception('API error')
    return 'Success!'

result = await retry_with_backoff(flaky_api_call)
print(f'Result: {result}')

## Circuit Breaker Pattern

In [ ]:
class CircuitBreaker:
    '''Circuit breaker to prevent cascading failures.'''
    
    def __init__(self, failure_threshold: int = 5, timeout: float = 60.0):
        self.failure_threshold = failure_threshold
        self.timeout = timeout
        self.failures = 0
        self.last_failure_time = None
        self.state = 'closed'  # closed, open, half-open
    
    def call(self, func):
        '''Call function with circuit breaker.'''
        if self.state == 'open':
            raise Exception('Circuit breaker is open')
        
        try:
            result = func()
            self.on_success()
            return result
        except Exception as e:
            self.on_failure()
            raise
    
    def on_success(self):
        self.failures = 0
        self.state = 'closed'
    
    def on_failure(self):
        self.failures += 1
        if self.failures >= self.failure_threshold:
            self.state = 'open'
            print(f'Circuit breaker opened after {self.failures} failures')

## Best Practices

1. **Always retry transient errors** (rate limits, timeouts)
2. **Use exponential backoff** (not linear)
3. **Set max retries** (don't retry forever)
4. **Log all errors** (for debugging)
5. **Implement circuit breakers** (for cascading failures)